# Практическое задание 2 (часть 2)

# Распознавание именованных сущностей из Twitter с помощью LSTM

## курс "Математические методы анализа текстов"


### ФИО: <впишите>

## Введение

### Постановка задачи

В этом задании вы будете использовать рекуррентные нейронные сети для решения проблемы распознавания именованных сущностей (NER). Примерами именованных сущностей являются имена людей, названия организаций, адреса и т.д. В этом задании вы будете работать с данными twitter.

Например, вы хотите извлечь имена и названия организаций. Тогда для текста

    Yan Goodfellow works for Google Brain

модель должна извлечь следующую последовательность:

    B-PER I-PER    O     O   B-ORG  I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система введена, чтобы различать последовательные именованные сущности одного типа.

Решение этого задания будет основано на нейронных сетях, а именно на Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Numpy](http://www.numpy.org).
 
### Данные

Все данные содержатся в папке `./data`: `./data/train.txt`, `./data/validation.txt`, `./data/test.txt`.

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1s1rFOFMZTBqtJuQDcIvW-8djA78iUDcx)

In [ ]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

## Часть 1. Подготовка данных (2 балла)

### Загрузка данных

Мы будем работать с данными, которые содержат твиты с тегами именованных сущностей. Каждая строка файла содержит пару токен(слово или пунктуация) и тег, разделенные пробелом. Различные твиты разделены пустой строкой.

Функция *read_data* считывает корпус из *file_path* и возвращает два списка: один с токенами и один с соответствующими токенам тегами. Также она заменяет все ники (токены, которые начинаются на символ *@*) на токен `<USR>` и url-ы (токены, которые начинаются на *http://* или *https://*) на токен `<URL>`. 

Вам необходимо реализовать эту функцию.

In [ ]:
def read_data(file_path):
    tokens = []
    tags = []

    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    return tokens, tags

Теперь мы можем загрузить 3 части данных:
 - *train* для тренировки модели;
 - *validation* для валидации и подбора гиперпараметров;
 - *test* для финального тестирования.

In [ ]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

Всегда полезно знать, с какими данными вы работаете. Выведем небольшую часть.

In [ ]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения.

- {token}$\to${token id}: устанавливает соответствие между токеном и строкой в embedding матрице;
- {tag}$\to${tag id}: one hot encoding тегов.


Теперь вам необходимо реализовать функцию *build_dict*, которая должна возвращать словарь {token or tag}$\to${index} и контейнер, задающий обратное отображение.

In [ ]:
from collections import defaultdict

In [ ]:
def build_dict(tokens_or_tags, special_tokens):
    """
    tokens_or_tags: a list of lists of tokens or tags
    special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    # Create mappings from tokens to indices and vice versa
    # Add special tokens to dictionaries
    # The first special token must have index 0

    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    return tok2idx, idx2tok

После реализации функции *build_dict* вы можете создать словари для токенов и тегов. В нашем случае специальными токенами будут:
 - `<UNK>` токен для обозначаения слов, которых нет в словаре;
 - `<PAD>` токен для дополнения предложений одного батча до одинаковой длины.

In [ ]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

### Генератор батчей

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины. Для некоторых фреймворков (таких как tensorflow) это необходимо сделать до подачи батча в нейронную сеть. В случае с pytorch это можно сделать как вне архитектуры нейронной сети, так и внутри. Мы выбрали более универсальный вариант и наш генератор батчей дополняет все последовательности внутри одного батча до одной длины.

Генератор батчей разбивает последовательность входных предложений и тегов на батчи размера batch_size. Размер последнего батча может быть меньше, если allow_smaller_last_batch is True, иначе последний батч исключается из генератора. Если включён параметр shuffle, данные перед разделением на батчи будут перемешаны. 

In [ ]:
def batches_generator(batch_size, tokens_idxs, tags_idxs,
                      shuffle=True, allow_smaller_last_batch=True, device='cpu'):
    """
    Generates padded batches of tags_idxs and tags_idxs.
    
    batch_size : int, number of objects in one batch
    tokens_idxs : list of list of int
    tags_idxs : list of list of int
    shuffle : bool
    allow_smaller_last_batch : bool
    device: str, cpu or cuda:x
    
    yield x, y: torch.LongTensor and torch.LongTensor
    x - batch of tokens_idxs, y - batch of tags_idxs
    """
    n_samples = len(tokens_idxs)
    
    # Shuffle data if shuffle is True.
    # Don't modify the original tokens and tags!
    
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    # Get the number of batches
    # n_batches =  ### YOUR CODE HERE ###
    
    # For each k yield pair x and y
    for k in range(n_batches):
        
        ######################################
        ######### YOUR CODE HERE #############
        ######################################        
        
        x = x.long().to(device)
        y = y.long().to(device)
        
        yield x, y

Протестируйте ваш генератор батчей:

In [ ]:
test_nonrandom_batch_generator = batches_generator(
    batch_size=3,
    tokens_idxs=train_idxs[:7],
    tags_idxs=train_tag_idxs[:7],
    shuffle=False,
    allow_smaller_last_batch=True
)

batch_lengths = [3, 3, 1]
sequence_lengths = [26, 25, 8]
some_pad_tensor = torch.LongTensor([token2idx['<PAD>']] * 12)
some_outside_tensor = torch.LongTensor([tag2idx['O'] * 12])

for i, (tokens_batch, tags_batch) in enumerate(test_nonrandom_batch_generator):
    assert tokens_batch.dtype == torch.int64, 'tokens_batch is not LongTensor'
    assert tags_batch.dtype == torch.int64, 'tags_batch is not LongTensor'
    
    assert len(tokens_batch) == batch_lengths[i], 'wrong batch length'
    
    for one_token_sequence in tokens_batch:
        assert len(one_token_sequence) == sequence_lengths[i], 'wrong length of sequence in batch'
    
    if i == 0:
        assert torch.all(tokens_batch[2][-12:] == some_pad_tensor), "wrong padding"       
        assert torch.all(tags_batch[2][-12:] == some_outside_tensor), "wrong O tag"

## Часть 2. BiLSTM (3 балла)

Определите архитектуру сети, используя библиотеку pytorch. 

**Замечания:**
1. Для улучшения качества сети предлагается использовать дополнительный Embedding слой на входе (каждому слову ставится в соответствие обучаемый вектор). 

2. Не забудьте, что `<PAD>` токены не должны учавствовать в подсчёте функции потерь.

In [ ]:
class BiLSTMModel(torch.nn.Module):
    def __init__(self, vocabulary_size, n_tags, PAD_index,
                 embedding_dim, rnn_hidden_size,
                 dropout_zeroed_probability,
                 device='cpu'):
        '''
        Defines neural network structure.
        
        architecture: input -> Embedding -> BiLSTM -> Dropout -> Linear
        optimizer: Adam
        
        ----------
        Parameters
        
        vocabulary_size: int, number of words in vocabulary.
        n_tags: int, number of tags.
        PAD_index: int, index of padding character. Used for loss masking.
        embedding_dim: int, dimension of words' embeddings.
        rnn_hidden_size: int, number of hidden units in each LSTM cell
        dropout_zeroed_probability: float, dropout zeroed probability for Dropout layer.
        device: str, cpu or cuda:x
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################        

        
    def forward(self, x_batch):
        '''
        Makes forward pass.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################        
    
    def predict_for_batch(self, x_batch):
        '''
        Returns predictions for x_batch.
        
        return type: torch.LongTensor
        return shape: (number of samples in batch, number words in sentence.
        
        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################        
    
    def train_on_batch(self, x_batch, y_batch, optimizer, loss_function):
        '''
        Trains model on the given batch.
        
        ----------
        Parameters
        x_batch: np.ndarray with shape (number of samples in batch, number words in sentence).
        y_batch: np.ndarray with shape (number of samples in batch).
        optimizer: torch.optimizer class
        loss_function: torch loss class
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################        


Для тестирования сети мы подготовили для вас две функции:
 - *predict_tags*: получает батч данных и трансформирует его в список из токенов и предсказанных тегов;
 - *eval_conll*: вычисляет метрики precision, recall и F1

In [ ]:
from evaluation_ner import precision_recall_f1

In [ ]:
def predict_tags(model, token_idxs_batch):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(token_idxs_batch)
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            if token_idx != token2idx['<PAD>']:
                tags.append(idx2tag[tag_idx])
                tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, x_batch)
        ground_truth_tags = [idx2tag[tag_idx] for tag_idx in y_batch[0]]

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(tags_batch[0] + ['O'])
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

### Эксперименты

Задайте BiLSTMModel. Рекомендуемые параметры:
- *batch_size*: 32;
- начальное значение *learning_rate*: 0.01-0.001
- *dropout_zeroed_probability*: 0.7-0.9
- *embedding_dim*: 100-200
- *rnn_hidden_size*: 150-200

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1 мере на валидационной и тестовой выборках было не меньше 0.35. 

Если сеть плохо обучается, попробуйте использовать следующие модификации:
    * используйте gradient clipping
    * на каждой итерации уменьшайте learning rate (например, в 1.1 раз)
    * попробуйте вместо Adam другие оптимизаторы 
    * экспериментируйте с dropout

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################        

## Бонусная часть. Улучшение качества теггера (4 балла).

Улучшите качество теггера на данной задаче.

Бонусные баллы будут начисляться в зависимости от результата f1-меры (одновременно на тестовой и валидационной выборках!).

+ 1 балл — $> 0.38$
+ 2 балла — $> 0.4$
+ 3 балла — $> 0.425$
+ 4 балла — $> 0.45$

Разрешается использовать любые разумные способы (в том числе и не рассматривающиеся в курсе). Под неразумными способами понимаются любые, в которых используются модели, обученные на dev или test, а также модели, использующие утечки в данных, не относяющиеся к смыслу задачи.

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################        